In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel
import requests
import responses
from PIL import Image
import io
from ipywidgets import widgets, VBox, HBox, Box, HTML, Layout,interact
import json

In [2]:
df = pd.read_csv('movies.csv')
movies =df.copy()
#movies.head()
#print('shape of movies dataset',movies.shape)


In [3]:
#movies.columns

In [4]:
#movies['id'].head()

In [5]:
df = pd.read_csv('credits.csv', usecols=['movie_id','title'])
credits = df.copy()
credits= credits.rename(columns={'movie_id':'id'},index=str)
#print('shape of credit dataset',credits.shape)


In [6]:
df = pd.read_csv('credits.csv')
credits = df.copy()
credits= credits.rename(columns={'movie_id':'id'},index=str)
#credits.head()
#print('shape of credit dataset',credits.shape)
  

In [7]:
merged_df = movies.merge(credits,on='id')
#merged_df .head(1)

In [8]:
merged_df = merged_df.drop(['homepage','title_x','title_y','status','production_companies'],axis=1)

In [9]:
#merged_df.info()

In [10]:
#merged_df.head(1)['overview']
#print(merged_df['overview'].isnull().sum())

In [11]:
#merged_df.columns

In [12]:
using_value_of_merged_df= merged_df[['id','original_title','overview','vote_average']]
#print(using_value_of_merged_df.shape)
#using_value_of_merged_df

In [13]:
indices = pd.Series(movies.index,index = using_value_of_merged_df['original_title'])
#type(indices)
indices = indices.drop_duplicates()
#print(indices.shape)
#indices

In [14]:
indices = pd.DataFrame(indices).reset_index()

In [15]:
indices= indices.rename(columns={0:'index_values'},index=str)
#indices

In [16]:
final_df = indices.merge(using_value_of_merged_df,on='original_title')
#final_df['index_values'] = final_df['index_values'].apply(str)
#final_df['id'] = final_df['id'].apply(str)
#final_df

In [17]:

tvf = TfidfVectorizer(min_df=3,
                      max_features=None,
                      strip_accents='unicode',
                      analyzer='word',
                      token_pattern=r'\w{1,}',
                      ngram_range=(1,3),
                      stop_words='english')

#imputing Nan values with empty strings

final_df['overview'] = final_df['overview'].fillna('')


In [18]:
#print(final_df['overview'].isnull().sum())

In [19]:
tvf_matrix = tvf.fit_transform(final_df['overview'])
#print('tvf_matrix',tvf_matrix)
#print('Shape of tvf_matrix', tvf_matrix.shape)

In [20]:
sigmod = sigmoid_kernel(tvf_matrix,tvf_matrix)

In [21]:
#final_df.info()

In [22]:
def recommendation(title,sig=sigmod):
    
        
    final_val = final_df.loc[final_df['original_title'] == title]
    index  = final_val['index_values']
    
    # final_val['index_values'] --> this is returning the values as dataframe which sigmond fun is not accepting it accepts just integer value.
    # Converting pandas df value into list and fetcing value of zero index and passing to sigmod
    index = list(index)[0]
    sigmod_values = sigmod[index]

    sig_scores =  list(enumerate(sigmod_values)) 
    #print(sig_scores) 
        
        
    sig_scores = sorted(sig_scores,key = lambda x:x[1],reverse=True)
    sig_scores =sig_scores[1:13]
    
    #for movies in sig_scores:
    movies_indices = [i[0] for i in sig_scores]
    #print('movies_indice',movies_indices)
    
    return final_df[['original_title','id','vote_average']].iloc[movies_indices] 

In [23]:
result_df = ""
def returnRecommendedMovies(title):
    global result_df
    result_df = recommendation(title)
    return pd.DataFrame(result_df)

In [24]:
returnRecommendedMovies('Big Daddy')


,original_title,id,vote_average
2530,National Lampoon's Vacation,11153,7.1
1197,Horrible Bosses 2,227159,6.1
28,Jurassic World,135397,6.5
1960,Dangerous Liaisons,859,7.0
4409,Karachi se Lahore,357441,8.0
675,Jurassic Park,329,7.6
3484,Miss March,19556,5.4
2836,Johnson Family Vacation,19084,4.7
3359,Flirting with Disaster,2074,6.3
4308,Blue Like Jazz,98369,5.8


In [25]:
result_df

,original_title,id,vote_average
2530,National Lampoon's Vacation,11153,7.1
1197,Horrible Bosses 2,227159,6.1
28,Jurassic World,135397,6.5
1960,Dangerous Liaisons,859,7.0
4409,Karachi se Lahore,357441,8.0
675,Jurassic Park,329,7.6
3484,Miss March,19556,5.4
2836,Johnson Family Vacation,19084,4.7
3359,Flirting with Disaster,2074,6.3
4308,Blue Like Jazz,98369,5.8


In [26]:

menu = np.array(final_df['original_title'].values.tolist()) 
menu = np.unique(menu)

DropDown = widgets.Dropdown(options=menu,
    value='Spider-Man 2',
    description='<div style = "height:100px;width:70px;border-style:#;padding-bottom:10px;font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#999999;"><b>Movies</b></div> :</br></br>',
)

    
def highlight_col(x):
    original_title = 'background-color: #d5ff80'
    Ratings =        'background-color: #d5ff80'

    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 0] = original_title
    df1.iloc[:, 1] = Ratings
    return df1



def show_latest_cases(title): 
    #showDataFrame = returnRecommendedMovies(title)
    
    pd.options.display.float_format = '{:.1f}'.format
    showDataFrame = result_df.copy()
    showDataFrame = showDataFrame.drop(['id'],axis=1)
    showDataFrame= showDataFrame.rename(columns={'original_title':'Movies','vote_average':'Ratings'})
    th_props = [
         ('font-size', '20px'),
         ('text-align', 'center'),
         ('font-weight', 'bold'),
         ('color', '#999999'),
         ('background-color', '#f7f7f9')
     ]

    # Set table styles
    styles = [
        dict(selector="th", props=th_props)
    ]
    s = showDataFrame.style.apply(highlight_col, axis=None) 
    s = s.hide_index()
    s.set_table_styles(styles)
    s.set_properties(**{'border-style': 'solid',
                        'border-color': '#e0e0eb',
                        'border-width':'0.015px',
                        'color': 'black',
                        'font-family':'Trebuchet MS',
                        'font-weight':'bold',
                        'text-align':'center',
                        'width':'255px'
                           })
    s.set_precision(1)
    s = s.set_caption('<h2>TOP FIVE RECOMMENDED MOVIES RELATED TO TITLE</h2><br>')
    pd.option_context('display.colheader_justify','center')
    return s

#displayRow(box)
DropDown.layout.height = '40px'
#displayRecommendedMovieList()
interact(show_latest_cases, title=DropDown);



interactive(children=(Dropdown(description='<div style = "height:100px;width:70px;border-style:#;padding-botto…

In [27]:

api_key = 'XXXXXXXXXXXXXXXXXXXX'
language = 'en-US'

images = []
recommended_movie_titles = []
placeholder = Image.open("placeholder_desaturated.jpg")

returnRecommendedMovies(DropDown.get_interact_value())
for i in result_df['id'].head(12):
    movie_id = str(i)
    url_get_poster_path ='https://api.themoviedb.org/3/movie/'+movie_id+'?api_key='+api_key+'&language='+language+''
    response = requests.get(url_get_poster_path)
    #print(response.status_code)
    #print(response.content)
    

    # Get the response data as a python object. Verify that it's a dictionary.
    try:
        data = response.json()
        
        #print(type(data))
        movie_title = data.get('original_title')
        #print(movie_title)
        recommended_movie_titles.append(movie_title)
        poster_path = data.get('belongs_to_collection')['poster_path']
        #print(poster_path)

        if poster_path != 'None':
            url_get_poster ='https://image.tmdb.org/t/p/w200'+poster_path+''
            image = Image.open(requests.get(url_get_poster, stream=True).raw)
            images.append(image)
            #print(image)
    except Exception as e:
        #print (e)
        images.append(placeholder)
        #print ("")
        

In [28]:
print(len(images))

12


In [29]:
def VBox(*pargs, **kwargs):
    """Displays multiple widgets vertically using the flexible box model."""
    box = Box(*pargs, **kwargs)
    box.layout.display = 'flex'
    box.layout.flex_flow = 'column'
    box.layout.align_items = 'stretch'
    return box

def HBox(*pargs, **kwargs):
    """Displays multiple widgets horizontally using the flexible box model."""
    box = Box(*pargs, **kwargs)
    box.layout.display = 'flex'
    box.layout.align_items = 'stretch'
    return box


In [30]:
def make_box_for_grid(image_widget, fit):
    """
    Make a VBox to hold caption/image for demonstrating
    option_fit values.
    """
    # Make the caption
    fit_str = str(fit)

    h = HTML(value='' + str(fit_str) + '')

    # Make the green box with the image widget inside it
    boxb = Box()
    boxb.layout = Layout()
    boxb.children = [image_widget]

    # Compose into a vertical box
    vb = VBox()
    vb.layout.align_items = 'center'
    vb.children = [h, boxb]
    return vb

In [31]:
def displayRow(box):
    sidebyside = widgets.HBox(box)
    display(sidebyside)

In [32]:
widgetList = []
for image in images:
    byteIO = io.BytesIO()
    image.save(byteIO, format='JPEG')
    widgetList.append(widgets.Image(value=byteIO.getvalue(), width=200, height=300))

In [33]:
boxes = []
i = 0
#print ("Length widgetList: ", len(widgetList))
#print ("Length movieTitles: ", len(movie_titles))
while(i < len(recommended_movie_titles)):
    boxes.append(make_box_for_grid(widgetList[i], recommended_movie_titles[i]))
    i= i+1

In [34]:
box1 = boxes[0:4]
box2 = boxes[4:8]
box3 = boxes[8::]

displayRow(box1)
displayRow(box2)
displayRow(box3)

In [80]:
# ! pip install requests
# ! pip install responses